TODO

- adjust links according to your file system
- check the existing folders to save the files

# Imports

In [1]:
import pandas as pd

from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import numpy as np
import numpy.ma as ma

from PIL import Image

import plotly.graph_objects as go
import plotly.subplots as sp

import peakutils 

import scipy
from scipy.signal import find_peaks, butter, filtfilt
import scipy.io
from scipy.ndimage import gaussian_filter
from scipy.stats import norm

import os

import tifffile as tiff
from tifffile import imshow

from ipywidgets import interactive, fixed, IntSlider, Checkbox

import skimage.io as skio
from skimage import exposure
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.filters import try_all_threshold
from skimage.filters import threshold_triangle, threshold_li, gaussian
from skimage.morphology import remove_small_objects, binary_dilation, binary_erosion, label
from skimage.color import label2rgb

import cv2

import plotly.express as px

from ipywidgets import interactive, IntSlider
from IPython.display import display



# Functions

In [76]:
def cfu_to_tif(cfu_link, cfu_global, path_to_save):
    """
    Converts a CFU MATLAB data to a TIFF image and saves it.
    """
    mat = scipy.io.loadmat(cfu_link)
    os.makedirs(path_to_save, exist_ok=True)
    cfu_i = mat[cfu_global]
    cfu_info_normalized = (cfu_i / np.max(cfu_i) * 255).astype(np.uint8)
    path_to_save = os.path.join(path_to_save, cfu_global + '.tiff')
    image = Image.fromarray(cfu_info_normalized)
    image.save(path_to_save)
    return path_to_save

def cfu_to_tif_global(cfu_link, path_to_save, cfu_global = 'cfuInfo1', index_2=2):
    """
    Converts a CFU MATLAB data to a TIFF image and saves it.
    """
    mat = scipy.io.loadmat(cfu_link)
    os.makedirs(path_to_save, exist_ok=True)
    df_test = {}
    mask_name = 'Mask_'
    for i in range(mat[cfu_global].shape[0]):
        a = Image.fromarray((mat['cfuInfo1'][i][index_2] / np.max( mat['cfuInfo1'][i][index_2]) * 255).astype(np.uint8))
        df_test['Mask_' + str(i)] = a
        path = os.path.join(path_to_save, cfu_global + mask_name + str(i) + '.tiff')
        image = a
        image.save(path)
    return 
def get_data_global(cfu_link, path_to_save, cfu_global = 'cfuInfo1'):
    """
    Converts a CFU MATLAB data to a TIFF image and saves it.
    """
    mat = scipy.io.loadmat(cfu_link)
    os.makedirs(path_to_save, exist_ok=True)
    df_data = {}
    df_data_df = {}
    mask_name = 'Mask_'
    for i in range(mat[cfu_global].shape[0]):
        df_data['Mask_' + str(i)] = mat['cfuInfo1'][i][4][0]
        df_data_df['Mask_' + str(i)] = mat['cfuInfo1'][i][5][0]
    #df_data = pd.DataFrame(df_data)
    #df_data_df = pd.DataFrame(df_data_df)
    return df_data, df_data_df
def df_trans_ren(df, global_coef):
    """
    Transposes a DataFrame and renames columns with a prefix.
    """
    df_transposed = df.T
    df_transposed.columns = [global_coef + f"{i+1}" for i in range(df_transposed.shape[1])]
    return df_transposed


def calculate_baseline(df, sigma=10, deg=5):
    """
    Calculates baselines for each column in the DataFrame.
    """
    baselines = {}
    for col in df.columns:
        smoothed_data = gaussian_filter(df[col].values, sigma=sigma)
        baselines[col] = peakutils.baseline(smoothed_data, deg=deg)
    return pd.DataFrame(baselines)

def plot_and_save(df, baselines, output_dir, combined_filename="all_cfus_combined_graph.html"):
    """
    Plots and saves individual CFU graphs and a combined graph.
    """
    os.makedirs(output_dir, exist_ok=True)
    fig_combined = go.Figure()
    for col in df.columns:
        cfu_data = df[col].values
        baseline = baselines[col].values
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=cfu_data, mode='lines', name=f"CFU {col}", line=dict(color="blue")))
        fig.add_trace(go.Scatter(y=baseline, mode='lines', name=f"Baseline {col}", line=dict(color="red", dash="dash")))
        fig.update_layout(
            title=f"Graph for CFU {col} with Baseline",
            xaxis_title="Index",
            yaxis_title="Intensity",
            template="plotly_white"
        )
        output_path = os.path.join(output_dir, f"cfu_{col}_graph.html")
        fig.write_html(output_path)
        print(f"Saved interactive graph for CFU {col} as {output_path}")
        fig_combined.add_trace(go.Scatter(y=cfu_data, mode='lines', name=f"CFU {col}"))
        fig_combined.add_trace(go.Scatter(y=baseline, mode='lines', name=f"Baseline {col}", line=dict(dash="dash")))
    combined_output_path = os.path.join(output_dir, combined_filename)
    fig_combined.update_layout(
        title="Combined Graph for All CFUs with Baselines",
        xaxis_title="Index",
        yaxis_title="Intensity",
        template="plotly_white"
    )
    fig_combined.write_html(combined_output_path)
    print(f"Saved combined graph as {combined_output_path}")

def detect_peaks(df, height=None, prominence=None, distance=None):
    """
    Detects peaks in each column of a DataFrame.
    
    Parameters:
        df (pd.DataFrame): DataFrame with Delta F/F data.
        height (float, optional): Minimum height of peaks.
        prominence (float, optional): Minimum prominence of peaks.
        distance (int, optional): Minimum distance between peaks.
        
    Returns:
        dict: A dictionary with column names as keys and detected peak indices as values.
        pd.DataFrame: A DataFrame with the number of peaks detected for each CFU.
    """
    peak_indices = {}
    peak_counts = {}

    for col in df.columns:
        data = df[col].values
        peaks, _ = find_peaks(data, height=height, prominence=prominence, distance=distance)
        peak_indices[col] = peaks
        peak_counts[col] = len(peaks)
    peak_counts_df = pd.DataFrame(list(peak_counts.items()), columns=["CFU", "Peak Count"])
    peak_counts_df.set_index("CFU", inplace=True)
    
    return peak_indices, peak_counts_df

def plot_peaks(df, peak_indices, output_dir):
    """
    Plots data with detected peaks for each CFU.
    """
    os.makedirs(output_dir, exist_ok=True)

    for col in df.columns:
        data = df[col].values
        peaks = peak_indices[col]
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=data, mode='lines', name=f"CFU {col}"))
        fig.add_trace(go.Scatter(
            x=peaks, y=data[peaks], mode='markers', name=f"Peaks {col}",
            marker=dict(color='red', size=8)
        ))
        fig.update_layout(
            title=f"Detected Peaks for CFU {col}",
            xaxis_title="Index",
            yaxis_title="Intensity",
            template="plotly_white"
        )
        output_path = os.path.join(output_dir, f"cfu_{col}_peaks.html")
        fig.write_html(output_path)
        print(f"Saved peak detection graph for CFU {col} to {output_path}")

def low_pass_filter(data, cutoff, fs, order=4):
    """
    Apply a low-pass Butterworth filter to the data.

    Parameters:
        data (np.ndarray): The input data to filter.
        cutoff (float): The cutoff frequency in Hz.
        fs (float): The sampling frequency in Hz.
        order (int): The order of the filter.
    
    Returns:
        np.ndarray: The filtered data.
    """
    nyquist = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyquist  # Normalize cutoff frequency
    b, a = butter(order, normal_cutoff, btype='low', analog=False)  # Design filter
    filtered_data = filtfilt(b, a, data)  # Apply filter
    return filtered_data


def detect_peaks_on_smoothed_data(df, fs=30, cutoff=5, height=None, prominence=None, distance=None):
    """
    Detect peaks on smoothed data using a low-pass filter for noise reduction.

    Parameters:
        df (pd.DataFrame): DataFrame with original Delta F/F data.
        fs (float): Sampling frequency in Hz (default 30 Hz).
        cutoff (float): Cutoff frequency for the low-pass filter in Hz (default 5 Hz).
        height (float, optional): Minimum height of peaks.
        prominence (float, optional): Minimum prominence of peaks.
        distance (int, optional): Minimum distance between peaks.
        
    Returns:
        dict: A dictionary with column names as keys and detected peak indices as values.
        dict: A dictionary with column names as keys and smoothed data as values.
        pd.DataFrame: A DataFrame with the number of peaks detected for each CFU.
    """
    peak_indices = {}
    smoothed_data_dict = {}
    peak_counts = {}

    for col in df.columns:
        smoothed_data = low_pass_filter(df[col].values, cutoff=cutoff, fs=fs)
        smoothed_data_dict[col] = smoothed_data
        peaks, _ = find_peaks(smoothed_data, height=height, prominence=prominence, distance=distance)
        peak_indices[col] = peaks
        peak_counts[col] = len(peaks)
    peak_counts_df = pd.DataFrame(list(peak_counts.items()), columns=["CFU", "Peak Count"])
    peak_counts_df.set_index("CFU", inplace=True)
    
    return peak_indices, smoothed_data_dict, peak_counts_df

def plot_peaks_on_smoothed_data_with_baseline(df, smoothed_data_dict, peak_indices, baseline_dict, output_dir, fps):
    """
    Plots smoothed data, original df, baseline, and detected peaks for each CFU in seconds.

    Parameters:
        df (pd.DataFrame): Original data for reference.
        smoothed_data_dict (dict): Dictionary of smoothed data.
        peak_indices (dict): Dictionary of detected peaks.
        baseline_dict (dict): Dictionary of baseline values for each CFU.
        output_dir (str): Directory to save the plots.
        fps (float): Frames per second of the data.
    """
    os.makedirs(output_dir, exist_ok=True)
    fig_combined = go.Figure()

    for col in df.columns:
        original_data = df[col].values
        smoothed_data = smoothed_data_dict[col]
        peaks = peak_indices[col]
        baseline = baseline_dict[col]
        time_seconds = np.arange(len(original_data)) / fps  # Convert frame numbers to seconds

        # Create the plot
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=time_seconds, y=original_data, mode='lines', name=f"Original CFU {col}", line=dict(color="blue")))
        fig.add_trace(go.Scatter(x=time_seconds, y=smoothed_data, mode='lines', name=f"Smoothed CFU {col}", line=dict(color="green")))
        fig.add_trace(go.Scatter(x=time_seconds, y=baseline, mode='lines', name=f"Baseline {col}", line=dict(color="orange", dash='dash')))
        fig.add_trace(go.Scatter(
            x=time_seconds[peaks], y=smoothed_data[peaks], mode='markers', name=f"Peaks {col}",
            marker=dict(color='red', size=8)
        ))
        fig.update_layout(
            title=f"Smoothed Data with Detected Peaks for CFU {col}",
            xaxis_title="Time (s)",
            yaxis_title="Intensity",
            template="plotly_white"
        )

        output_path = os.path.join(output_dir, f"cfu_{col}_smoothed_peaks.html")
        fig.write_html(output_path)
        print(f"Saved peak detection graph for CFU {col} to {output_path}")

        # Add to combined plot
        fig_combined.add_trace(go.Scatter(x=time_seconds, y=smoothed_data, mode='lines', name=f"Smoothed CFU {col}"))
        fig_combined.add_trace(go.Scatter(
            x=time_seconds[peaks], y=smoothed_data[peaks], mode='markers', name=f"Peaks {col} ({len(peaks)})"
        ))
        fig_combined.add_trace(go.Scatter(x=time_seconds, y=baseline, mode='lines', name=f"Baseline {col} ({col})"))

    combined_output_path = os.path.join(output_dir, "all_cfus_smoothed_peaks_combined.html")
    fig_combined.update_layout(
        title="Smoothed Data with Detected Peaks for All CFUs",
        xaxis_title="Time (s)",
        yaxis_title="Intensity",
        template="plotly_white"
    )
    fig_combined.write_html(combined_output_path)
    print(f"Saved combined peak detection graph to {combined_output_path}")


def plot_peak_density_per_cfu(peak_indices, fs, output_dir="peak_density_plots", bins=30):
    """
    Plots the density of detected peaks per second for each CFU and saves as HTML.

    Parameters:
        peak_indices (dict): A dictionary with CFU names as keys and detected peak indices as values.
        fs (float): Sampling frequency in Hz.
        output_dir (str): Directory to save the HTML plots.
        bins (int): Number of bins for the histogram.
    """
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    for cfu, indices in peak_indices.items():
        # Convert peak indices to time (seconds)
        peak_times = np.array(indices) / fs

        # Create histogram bins
        peak_counts, bin_edges = np.histogram(peak_times, bins=bins)

        # Create the Plotly bar plot
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=bin_edges[:-1],
            y=peak_counts,
            width=(bin_edges[1] - bin_edges[0]),
            marker=dict(color="blue", opacity=0.7),
            name="Peak Density"
        ))

        # Update layout
        fig.update_layout(
            title=f"Peak Density for {cfu}",
            xaxis_title="Time (s)",
            yaxis_title="Peak Density (Peaks per Bin)",
            template="plotly_white",
            bargap=0.1
        )

        # Save the plot as HTML
        output_path = os.path.join(output_dir, f"peak_density_{cfu}.html")
        fig.write_html(output_path)
        print(f"Saved peak density plot for {cfu} to {output_path}")
        

def calculate_intensities(tiff_file, mask_folder, output_csv):
    """
    Calculate frame-wise intensities for each mask and save as a DataFrame.

    Parameters:
        tiff_file (str): Path to the multi-frame TIFF file.
        mask_folder (str): Path to the folder containing mask files as TIFFs.
        output_csv (str): Path to save the intensity DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing frame-wise intensities for each mask.
    """
    with tiff.TiffFile(tiff_file) as tif:
        tiff_data = tif.asarray()

    print(f"Loaded TIFF with shape {tiff_data.shape} (frames, height, width)")
    num_frames = tiff_data.shape[0]
    intensity_data = pd.DataFrame(index=range(num_frames))
    for mask_file in os.listdir(mask_folder):
        if not mask_file.endswith('.tiff'):
            continue
        mask_path = os.path.join(mask_folder, mask_file)
        with tiff.TiffFile(mask_path) as mask_tif:
            mask = mask_tif.asarray().astype(bool)
        if mask.shape != tiff_data.shape[1:]:
            raise ValueError(f"Mask {mask_file} shape {mask.shape} does not match TIFF shape {tiff_data.shape[1:]}")
        frame_intensities = [np.mean(frame[mask]) for frame in tiff_data]
        mask_name = os.path.splitext(mask_file)[0]
        intensity_data[mask_name] = frame_intensities
    intensity_data.to_csv(output_csv)
    print(f"Saved intensity data to {output_csv}")

    return intensity_data
    
def crop_tiff_dimensions(tiff_data, crop_pixels=5):
    """
    Crop the second and third dimensions (height and width) of a TIFF file.

    Parameters:
        tiff_data (np.ndarray): Original TIFF data (frames, height, width).
        crop_pixels (int): Number of pixels to crop from each side.

    Returns:
        np.ndarray: Cropped TIFF data.
    """
    return tiff_data[:, crop_pixels:-crop_pixels, crop_pixels:-crop_pixels]


def calculate_dff(intensity_data):
    """
    Calculate Delta F/F for the given intensity data using column-wise minimum as baseline.

    Parameters:
        intensity_data (pd.DataFrame): Frame-wise intensity data.

    Returns:
        pd.DataFrame: DataFrame containing Delta F/F values.
    """
    baselines = intensity_data.min(axis=0)
    dff = (intensity_data - baselines) / baselines
    return dff

'''
def detect_peaks(dff_data, height=None, prominence=None, distance=None):
    """
    Detect peaks in Delta F/F data for each mask.

    Parameters:
        dff_data (pd.DataFrame): Delta F/F DataFrame.
        height (float, optional): Minimum peak height.
        prominence (float, optional): Minimum prominence of peaks.
        distance (int, optional): Minimum distance between peaks.

    Returns:
        dict: A dictionary with column names as keys and detected peak indices as values.
    """
    peak_indices = {}

    for col in dff_data.columns:
        # Detect peaks
        peaks, _ = find_peaks(dff_data[col].values, height=height, prominence=prominence, distance=distance)
        peak_indices[col] = peaks

    return peak_indices
'''

def plot_dff_and_peaks(dff_data, peak_indices, output_dir):
    """
    Plot Delta F/F data and detected peaks for each mask.

    Parameters:
        dff_data (pd.DataFrame): Delta F/F DataFrame.
        peak_indices (dict): Dictionary of detected peak indices.
        output_dir (str): Directory to save the plots.
    """
    os.makedirs(output_dir, exist_ok=True)
    fig_combined = go.Figure()

    for col in dff_data.columns:
        dff_values = dff_data[col].values
        peaks = peak_indices[col]
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=dff_values, mode='lines', name=f"Delta F/F {col}", line=dict(color="blue")))
        fig.add_trace(go.Scatter(
            x=peaks, y=dff_values[peaks], mode='markers', name=f"Peaks {col}",
            marker=dict(color='red', size=8)
        ))
        fig.update_layout(
            title=f"Delta F/F and Peaks for {col}",
            xaxis_title="Frame",
            yaxis_title="Delta F/F",
            template="plotly_white"
        )
        fig.write_html(os.path.join(output_dir, f"{col}_dff_peaks.html"))
        fig_combined.add_trace(go.Scatter(y=dff_values, mode='lines', name=f"Delta F/F {col}"))
        fig_combined.add_trace(go.Scatter(
            x=peaks, y=dff_values[peaks], mode='markers', name=f"Peaks {col} ({len(peaks)})"
        ))
    combined_output_path = os.path.join(output_dir, "combined_dff_peaks.html")
    fig_combined.update_layout(
        title="Combined Delta F/F and Peaks for All Masks",
        xaxis_title="Frame",
        yaxis_title="Delta F/F",
        template="plotly_white"
    )
    fig_combined.write_html(combined_output_path)
    print(f"Saved combined Delta F/F and peaks plot to {combined_output_path}")

def plot_and_save_intensities(intensity_data, smoothed_data, peak_indices, output_dir):
    """
    Plot and save intensity and peak detection results.
    """
    os.makedirs(output_dir, exist_ok=True)
    fig_combined = go.Figure()

    for col in intensity_data.columns:
        original_data = intensity_data[col].values
        smoothed = smoothed_data[col]
        peaks = peak_indices[col]
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=original_data, mode='lines', name=f"Original {col}", line=dict(color="blue")))
        fig.add_trace(go.Scatter(y=smoothed, mode='lines', name=f"Smoothed {col}", line=dict(color="green")))
        fig.add_trace(go.Scatter(
            x=peaks, y=smoothed[peaks], mode='markers', name=f"Peaks {col}",
            marker=dict(color='red', size=8)
        ))
        fig.update_layout(
            title=f"Intensity and Peaks for {col}",
            xaxis_title="Frame",
            yaxis_title="Intensity",
            template="plotly_white"
        )
        fig.write_html(os.path.join(output_dir, f"{col}_peaks.html"))
        fig_combined.add_trace(go.Scatter(y=smoothed, mode='lines', name=f"Smoothed {col}"))
        fig_combined.add_trace(go.Scatter(
            x=peaks, y=smoothed[peaks], mode='markers', name=f"Peaks {col} ({len(peaks)})"
        ))
    combined_output_path = os.path.join(output_dir, "combined_peaks.html")
    fig_combined.update_layout(
        title="Combined Intensity and Peaks for All Masks",
        xaxis_title="Frame",
        yaxis_title="Intensity",
        template="plotly_white"
    )
    fig_combined.write_html(combined_output_path)
    print(f"Saved combined intensity and peaks plot to {combined_output_path}")

def save_cfu_analysis_with_report_v2(
    df, smoothed_data_dict, baseline_dict, peak_indices, output_dir,
    link_folder, link_file_mat, link_save,
    sigma_value, deg_value, fs_value, height_value, prominence_value, distance_value
):
    """
    Saves the CFU analysis results to CSV files and generates a detailed TXT report.

    Parameters:
        df (pd.DataFrame): Original CFU DataFrame.
        smoothed_data_dict (dict): Dictionary containing smoothed data for each CFU.
        baseline_dict (dict): Dictionary containing baseline data for each CFU.
        peak_indices (dict): Dictionary containing indices of detected peaks for each CFU.
        output_dir (str): Directory to save the CSV files and report.
        link_folder (str): Folder containing the data.
        link_file_mat (str): Path to the original MAT file.
        link_save (str): Folder where files are saved.
        sigma_value (float): Smoothing sigma value.
        deg_value (int): Degree of baseline correction.
        fs_value (float): Frames per second of the data.
        height_value (float): Minimum peak height for detection.
        prominence_value (float): Minimum peak prominence for detection.
        distance_value (int): Minimum distance between peaks.
    """
    os.makedirs(output_dir, exist_ok=True)
    cfu_analysis_df = pd.DataFrame()

    # Generate the main analysis DataFrame
    for cfu in df.columns:
        original_data = df[cfu]
        smoothed_data = smoothed_data_dict[cfu]
        baseline_data = baseline_dict[cfu]
        peak_mask = np.zeros(len(smoothed_data), dtype=int)
        peak_mask[peak_indices[cfu]] = 1  # Mark peaks as 1, others as 0

        cfu_data = pd.DataFrame({
            "CFU": cfu,
            "Original CFU Data": original_data,
            "Smoothed CFU Data": smoothed_data,
            "Baseline CFU Data": baseline_data,
            "Detected Peaks (Smoothed)": peak_mask
        })
        cfu_analysis_df = pd.concat([cfu_analysis_df, cfu_data], ignore_index=True)

    # Save the main CFU analysis DataFrame
    cfu_analysis_csv_path = os.path.join(output_dir, "cfu_analysis.csv")
    cfu_analysis_df.to_csv(cfu_analysis_csv_path, index=False)
    print(f"Saved CFU analysis to {cfu_analysis_csv_path}")

    # Generate the peaks summary DataFrame with counts
    peaks_data = []
    for cfu, indices in peak_indices.items():
        peaks_data.append({"CFU": cfu, "Detected Peak Count": len(indices)})

    peaks_df = pd.DataFrame(peaks_data)

    # Save the peaks summary DataFrame
    peaks_csv_path = os.path.join(output_dir, "detected_peaks_summary.csv")
    peaks_df.to_csv(peaks_csv_path, index=False)
    print(f"Saved detected peaks summary to {peaks_csv_path}")

    # Generate a report as a TXT file
    report_path = os.path.join(output_dir, "analysis_report.txt")
    with open(report_path, "w") as report_file:
        report_file.write("Analysis Report\n")
        report_file.write("=" * 40 + "\n\n")

        # Write metadata
        report_file.write("Links and Paths:\n")
        report_file.write(f"Data Folder: {link_folder}\n")
        report_file.write(f"MAT File: {link_file_mat}\n")
        report_file.write(f"Save Folder: {link_save}\n\n")

        # Write coefficients
        report_file.write("Coefficients Used:\n")
        report_file.write(f"Smoothing Sigma Value: {sigma_value}\n")
        report_file.write(f"Baseline Degree: {deg_value}\n")
        report_file.write(f"Frames Per Second (FPS): {fs_value}\n")
        report_file.write(f"Peak Detection Height: {height_value}\n")
        report_file.write(f"Peak Detection Prominence: {prominence_value}\n")
        report_file.write(f"Peak Detection Distance: {distance_value}\n\n")

        # Write FPS and DataFrame length
        report_file.write(f"Total Length of Data (Mask_0): {len(df['Mask_0'])}\n")

        # Write peaks summary
        report_file.write("\nNumber of Peaks per Mask:\n")
        for cfu, indices in peak_indices.items():
            report_file.write(f"- {cfu}: {len(indices)} peaks\n")

        # Write file paths
        report_file.write("\nFiles Saved:\n")
        report_file.write(f"- CFU Analysis: {cfu_analysis_csv_path}\n")
        report_file.write(f"- Peaks Summary: {peaks_csv_path}\n")

        # Add date and time of analysis
        report_file.write("\nDate and Time of Analysis:\n")
        report_file.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

        # Add output folder location
        report_file.write("\nOutput Folder:\n")
        report_file.write(f"{output_dir}\n")

    print(f"Saved analysis report to {report_path}")

def plot_combined_smoothed_and_peak_density(
    df, smoothed_data_dict, peak_indices, fps, height_value, prominence_value, distance_value, sigma_value, output_path
):
    """
    Plots the smoothed data with detected peaks, and aligns it with a barplot of peak density.

    Parameters:
        df (pd.DataFrame): Original Delta F/F data.
        smoothed_data_dict (dict): Dictionary of smoothed data for each CFU.
        peak_indices (dict): Dictionary of detected peaks for each CFU.
        fps (float): Sampling frequency in Hz.
        height_value (float): Minimum height of peaks for detection.
        prominence_value (float): Minimum prominence of peaks.
        distance_value (int): Minimum distance between peaks.
        sigma_value (float): Smoothing coefficient.
        output_path (str): Path to save the combined plot.
    """
    for cfu in df.columns:
        # Original data, smoothed data, and peaks
        original_data = df[cfu].values
        smoothed_data = smoothed_data_dict[cfu]
        peaks = peak_indices[cfu]
        time_seconds = np.arange(len(original_data)) / fps  # Convert frame numbers to seconds

        # Calculate peak density
        peak_times = np.array(peaks) / fps
        bin_edges = np.arange(0, time_seconds[-1] + 1, 1)  # Align bins to time axis
        peak_counts, bin_edges = np.histogram(peak_times, bins=bin_edges)

        # Create subplots
        fig = sp.make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.1,
            row_heights=[0.7, 0.3],
            subplot_titles=(f"Smoothed Data with Peaks for {cfu}", "Peak Density")
        )

        # Add smoothed data and peaks to the first subplot
        fig.add_trace(
            go.Scatter(x=time_seconds, y=original_data, mode='lines', name="Original Data", line=dict(color="blue", width=1)),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=time_seconds, y=smoothed_data, mode='lines', name="Smoothed Data", line=dict(color="green", width=2)),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=time_seconds[peaks], y=smoothed_data[peaks], mode='markers', name="Detected Peaks",
                       marker=dict(color='red', size=8)),
            row=1, col=1
        )

        # Add barplot of peak density to the second subplot
        fig.add_trace(
            go.Bar(
                x=bin_edges[:-1], y=peak_counts, width=1,  # Ensure bar width aligns with bins
                name="Peak Density", marker=dict(color="blue", opacity=0.7)
            ),
            row=2, col=1
        )

        # Update layout
        fig.update_layout(
            title=f"Combined Smoothed Data and Peak Density for {cfu}",
            xaxis_title="Time (s)",
            yaxis=dict(title="Intensity"),
            yaxis2=dict(title="Peak Density (Peaks per Bin)"),
            template="plotly_white",
            legend_title="Input Parameters",
            annotations=[
                dict(
                    text=(
                        f"<b>Parameters:</b><br>"
                        f"FPS: {fps}<br>"
                        f"Height: {height_value}<br>"
                        f"Prominence: {prominence_value}<br>"
                        f"Distance: {distance_value}<br>"
                        f"Smoothing Sigma: {sigma_value}"
                    ),
                    x=1.1, y=1.2, showarrow=False, xref="paper", yref="paper", align="left"
                )
            ]
        )

        # Save the plot
        fig.write_html(output_path + f"/combined_smoothed_peak_density_{cfu}.html")
        print(f"Saved combined plot for {cfu} to {output_path}")

def compare_peak_times(peak_indices, tolerance_frames, fps, output_path):
    """
    Compares whether different masks (CFUs) have peaks at the same time within a given tolerance
    and generates a heatmap showing the overlap.

    Parameters:
        peak_indices (dict): A dictionary with CFU names as keys and detected peak indices as values.
        tolerance_frames (int): Tolerance around the peak for comparison (in frames).
        fps (float): Frames per second, used to annotate the heatmap with time in seconds.
        output_path (str): Path to save the heatmap HTML file.
    """
    cfus = list(peak_indices.keys())
    num_cfus = len(cfus)
    overlap_matrix = np.zeros((num_cfus, num_cfus), dtype=int)
    for i, cfu_1 in enumerate(cfus):
        for j, cfu_2 in enumerate(cfus):
            if i <= j:  # Only compute upper triangle (matrix is symmetric)
                peaks_1 = np.array(peak_indices[cfu_1])
                peaks_2 = np.array(peak_indices[cfu_2])
                for peak_1 in peaks_1:
                    matches = np.any(np.abs(peaks_2 - peak_1) <= tolerance_frames)
                    overlap_matrix[i, j] += matches
    overlap_matrix = overlap_matrix + overlap_matrix.T - np.diag(overlap_matrix.diagonal())
    overlap_df = pd.DataFrame(overlap_matrix, index=cfus, columns=cfus)
    fig = px.imshow(
        overlap_df,
        labels=dict(x="CFU", y="CFU", color="Peak Overlap Count"),
        x=cfus,
        y=cfus,
        color_continuous_scale="Viridis",
        title=f"Peak Overlap Heatmap (Tolerance: +/- {tolerance_frames} frames)"
    )
    time_tolerance = tolerance_frames / fps
    fig.add_annotation(
        text=f"Time Tolerance: +/- {time_tolerance:.2f} seconds",
        xref="paper", yref="paper",
        x=1.1, y=1.1, showarrow=False
    )
    heatmap_path = f"{output_path}\peak_overlap_heatmap.html"
    fig.write_html(heatmap_path)
    print(f"Saved heatmap to {heatmap_path}")

    return overlap_df

def plot_high_overlap_pairs(df, smoothed_data_dict, peak_indices, overlap_df, overlap_threshold, fps, output_dir):
    """
    Identifies CFU pairs with more than a specified overlap threshold and plots their data together,
    highlighting regions with overlapped peaks.

    Parameters:
        df (pd.DataFrame): Original CFU data.
        smoothed_data_dict (dict): Dictionary of smoothed data for each CFU.
        peak_indices (dict): Dictionary of detected peaks for each CFU.
        overlap_df (pd.DataFrame): Overlap percentage DataFrame.
        overlap_threshold (float): Threshold for overlap percentage to consider pairs.
        fps (float): Frames per second of the data.
        output_dir (str): Directory to save the plots.
    """
    os.makedirs(output_dir, exist_ok=True)
    high_overlap_pairs = [
        (cfu_1, cfu_2) for cfu_1 in overlap_df.index for cfu_2 in overlap_df.columns
        if cfu_1 != cfu_2 and overlap_df.loc[cfu_1, cfu_2] > overlap_threshold
    ]

    for cfu_1, cfu_2 in high_overlap_pairs:
        original_data_1 = df[cfu_1].values
        smoothed_data_1 = smoothed_data_dict[cfu_1]
        peaks_1 = np.array(peak_indices[cfu_1])

        original_data_2 = df[cfu_2].values
        smoothed_data_2 = smoothed_data_dict[cfu_2]
        peaks_2 = np.array(peak_indices[cfu_2])

        time_seconds = np.arange(len(original_data_1)) / fps  # Convert frame numbers to seconds
        overlapping_peaks = [
            peak_1 for peak_1 in peaks_1 if np.any(np.abs(peaks_2 - peak_1) <= 4)
        ]
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=time_seconds, y=original_data_1, mode='lines',
            name=f"Original {cfu_1}", line=dict(color="blue", width=1)
        ))
        fig.add_trace(go.Scatter(
            x=time_seconds, y=smoothed_data_1, mode='lines',
            name=f"Smoothed {cfu_1}", line=dict(color="green", width=2)
        ))
        fig.add_trace(go.Scatter(
            x=time_seconds[peaks_1], y=smoothed_data_1[peaks_1], mode='markers',
            name=f"Peaks {cfu_1}", marker=dict(color='red', size=8)
        ))
        fig.add_trace(go.Scatter(
            x=time_seconds, y=original_data_2, mode='lines',
            name=f"Original {cfu_2}", line=dict(color="purple", width=1)
        ))
        fig.add_trace(go.Scatter(
            x=time_seconds, y=smoothed_data_2, mode='lines',
            name=f"Smoothed {cfu_2}", line=dict(color="orange", width=2)
        ))
        fig.add_trace(go.Scatter(
            x=time_seconds[peaks_2], y=smoothed_data_2[peaks_2], mode='markers',
            name=f"Peaks {cfu_2}", marker=dict(color='black', size=8)
        ))
        for overlap_peak in overlapping_peaks:
            overlap_time = time_seconds[overlap_peak]
            fig.add_vrect(
                x0=overlap_time - (4 / fps), x1=overlap_time + (4 / fps),
                fillcolor="rgba(255, 0, 0, 0.3)", layer="below", line_width=0
            )
        fig.update_layout(
            title=f"High Overlap Between {cfu_1} and {cfu_2} (> {overlap_threshold}%)",
            xaxis_title="Time (s)",
            yaxis_title="Intensity",
            template="plotly_white"
        )
        output_path = os.path.join(output_dir, f"overlap_{cfu_1}_{cfu_2}.html")
        fig.write_html(output_path)
        print(f"Saved high-overlap plot for {cfu_1} and {cfu_2} to {output_path}")
def bin_dataframe(df, bin_factor=2):
    """
    Bins a dataframe by averaging consecutive rows in groups of bin_factor.

    Parameters:
        df (pd.DataFrame): Input dataframe where rows represent frames and columns represent measurements (e.g., CFUs).
        bin_factor (int): Number of consecutive rows to average.

    Returns:
        pd.DataFrame: Binned dataframe with reduced row count.
    """
    num_rows = df.shape[0]
    if num_rows % bin_factor != 0:
        print(f"Warning: Number of rows ({num_rows}) is not divisible by bin_factor ({bin_factor}). "
              f"Truncating to {num_rows - (num_rows % bin_factor)} rows.")
        df = df.iloc[:num_rows - (num_rows % bin_factor)]
    binned_data = (
        df.values.reshape(-1, bin_factor, df.shape[1])
        .mean(axis=1)
    )
    binned_df = pd.DataFrame(binned_data, columns=df.columns)

    return binned_df



# Links

In [ ]:
link_folder = r'E:\CM006_uni4_test_matlab_aqua_CFUs\All_CFUs.mat'
link_save = r'E:\CM006_uni4_test_matlab_aqua_CFUs\Save'
link_file = r'E:\CM006_uni4_test_matlab_aqua_CFUs\unit4_crop4_1.tif'

# Data analysis

Data preparation

In [ ]:
mat = scipy.io.loadmat(link_folder)

In [ ]:
link_base = r'E:\CM006_uni4_test_matlab_aqua_CFUs\CFU_'
links_list = []
cfu_base = r'cfuInfo1'
cfu_list = []
path_to_s = r'E:\CM006_uni4_test_matlab_aqua_CFUs\path_'
for i in range(9):
    links_list.append(link_base + str(i+1))
    cfu_list.append(cfu_base + str(i+1))
print('CFU list: ' + str(cfu_list))
print('links list: ' + str(links_list))

In [ ]:
for i in range(9):
    cfu_to_tif(links_list[i], cfu_list[i], path_to_s)

Save all plotted intensities from the All_CFUs file

In [ ]:
all_cfus = scipy.io.loadmat(link_folder)['All_CFUs']
output_dir = link_save + '\All_CFUs_'  # Set your desired output directory

for i, row in enumerate(all_cfus):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=row, mode='lines', name=f"Row {i + 1}"))
    fig.update_layout(
        title=f"Graph for Row {i + 1}",
        xaxis_title="Index",
        yaxis_title="Intensity",
        template="plotly_white"
    )
    output_path = output_dir + f"row_{i + 1}_graph.html"
    fig.write_html(output_path)
    print(f"Saved interactive graph for row {i + 1} as {output_path}")

fig_combined = go.Figure()
for i, row in enumerate(all_cfus):
    fig_combined.add_trace(go.Scatter(y=row, mode='lines', name=f"Row {i + 1}"))
fig_combined.update_layout(
    title="All Rows Combined",
    xaxis_title="Index",
    yaxis_title="Intensity",
    template="plotly_white"
)

# Save the combined graph
combined_output_path = output_dir + "all_rows_combined_graph.html"
fig_combined.write_html(combined_output_path)
print(f"Saved combined interactive graph as {combined_output_path}")

make the CFUs data from matlab as dataframe

In [ ]:
df_all_cfus = pd.DataFrame(scipy.io.loadmat(r'E:\CM006_uni4_test_matlab_aqua_CFUs\All_CFUs.mat')['All_CFUs'])
df_all_cfus_df = pd.DataFrame(scipy.io.loadmat(r'E:\CM006_uni4_test_matlab_aqua_CFUs\All_CFUs_df.mat')['All_cfu_df'])
df_all_cfus = df_trans_ren(df_all_cfus, 'CFU_')
df_all_cfus_df = df_trans_ren(df_all_cfus_df, 'CFU_DF_')
df_all_cfus_bg_corr = df_all_cfus-110

In [ ]:
df_all_cfus_baseline = calculate_baseline(df_all_cfus, sigma=10, deg=5)
df_all_cfus_df_baseline = calculate_baseline(df_all_cfus_df, sigma=10, deg=5)
df_all_cfus_bg_corr_baseline = calculate_baseline(df_all_cfus_bg_corr, sigma=10, deg=5)

In [ ]:
df_all_cfus_bg_corr_baseline

In [ ]:
df_all_cfus_bg_corr_df =pd.DataFrame()
for col in df_all_cfus_bg_corr.columns:
    df_all_cfus_bg_corr_df[col] = (df_all_cfus_bg_corr[col]-df_all_cfus_bg_corr_baseline[col])/df_all_cfus_bg_corr_baseline[col]

df_all_cfus_bg_corr_df_baseline = calculate_baseline(df_all_cfus_bg_corr_df, sigma=10, deg=5)

In [ ]:
df_all_cfus_bg_corr

In [ ]:
plot_and_save(df_all_cfus_bg_corr, df_all_cfus_bg_corr_baseline, link_save + '\corrected_baseline')

In [ ]:
plot_and_save(df_all_cfus_bg_corr_df, df_all_cfus_bg_corr_df_baseline, link_save + '\corrected_df_baseline')

data from tif file through masks

# Peak detection

In [ ]:
peak_indices_df_aqua, peak_counts_df_aqua = detect_peaks(
    df=df_all_cfus_df,
    height=0.1,        # Minimum height of peaks (adjust as needed)
    prominence=0.08,   # Minimum prominence of peaks
    distance=30        # Minimum distance between peaks
)

print(peak_counts_df_aqua)
plot_peaks(df_all_cfus_df, peak_indices_df_aqua, link_save + '\CFU_Peaks_fromaqua')

In [ ]:
peak_indices, peak_counts_df = detect_peaks(
    df=df_all_cfus_bg_corr_df,
    height=0.1,        # Minimum height of peaks (adjust as needed)
    prominence=0.08,   # Minimum prominence of peaks
    distance=30        # Minimum distance between peaks
)

print(peak_counts_df)


In [ ]:
plot_peaks(df_all_cfus_bg_corr_df, peak_indices, link_save + '\CFU_Peaks')


In [ ]:
output_dir = r"E:\CM006_uni4_test_matlab_aqua_CFUs\CFU_Smoothed_Peaks"
peak_indices, smoothed_data_dict, peak_counts_df = detect_peaks_on_smoothed_data(
    df=df_all_cfus_bg_corr_df,  # Replace with your DataFrame
    sigma=2,
    height=0.1,
    prominence=0.08,
    distance=30
)
plot_peaks_on_smoothed_data(df_all_cfus_bg_corr_df, smoothed_data_dict, peak_indices, output_dir)
peak_counts_df.to_csv(os.path.join(output_dir, "peak_counts.csv"))
np.save(os.path.join(output_dir, "peak_indices.npy"), peak_indices)
print(f"Peak counts saved to {os.path.join(output_dir, 'peak_counts.csv')}")
print(f"Peak indices saved to {os.path.join(output_dir, 'peak_indices.npy')}")

In [ ]:
tiff_file = r"E:\CM006_uni4_test_matlab_aqua_CFUs\unit4_crop4_1.tif"
mask_folder = r"E:\CM006_uni4_test_matlab_aqua_CFUs\Masks"
output_csv = link_save + "\intensity_data.csv"
output_dir = r"E:\CM006_uni4_test_matlab_aqua_CFUs\Save\from_tif_dff"

cropped_tiff_file = r"E:\CM006_uni4_test_matlab_aqua_CFUs\unit4_crop4_1_cropped_file.tiff"
with tiff.TiffFile(tiff_file) as tif:
    tiff_data = tif.asarray()
print(f"Original TIFF shape: {tiff_data.shape}")
tiff_data_cropped = crop_tiff_dimensions(tiff_data, crop_pixels=5)
print(f"Cropped TIFF shape: {tiff_data_cropped.shape}")

tiff.imwrite(cropped_tiff_file, tiff_data_cropped)
print(f"Cropped TIFF saved to {cropped_tiff_file}")

intensity_data = calculate_intensities(cropped_tiff_file, mask_folder, output_csv)
intensity_data_corr = pd.DataFrame()
for col in intensity_data.columns:
    intensity_data_corr[col] = intensity_data[col] - intensity_data[col].min() +50
dff_data = calculate_dff(intensity_data_corr)
peak_indices, peak_counts_df = detect_peaks(dff_data, height=0.1, prominence=0.05, distance=20)
plot_dff_and_peaks(dff_data, peak_indices, output_dir)


In [ ]:
peak_indices, peak_counts_df = detect_peaks(
    df=dff_data,
    height=0.1,        # Minimum height of peaks (adjust as needed)
    prominence=0.1,   # Minimum prominence of peaks
    distance=50        # Minimum distance between peaks
)

print(peak_counts_df)

In [ ]:
frame_rate = 30.0  # Example: 30 frames per second
all_peaks = []
for roi, indices in peak_indices.items():
    all_peaks.extend(indices)
peak_times = np.array(all_peaks) / frame_rate
time_bins = np.arange(0, peak_times.max() + 1, 1)  # From 0 to max peak time, in 1-second bins
peak_counts, bin_edges = np.histogram(peak_times, bins=time_bins)
density_df = pd.DataFrame({
    "Time (s)": bin_edges[:-1],  # Start of each bin
    "Peak Density": peak_counts
})
plt.bar(density_df["Time (s)"], density_df["Peak Density"], width=1, align='edge', color='blue', alpha=0.7)
plt.xlabel("Time (s)")
plt.ylabel("Peak Density (Peaks per Second)")
plt.title("Density of Detected Peaks per Second")
plt.show()


# Get masks from mat file

In [ ]:
link_folder = r'E:\Data\241105_CM006\unit4'
link_file_mat = r'E:\Data\241105_CM006\unit4\unit4_32_crop_2_AQuA2.mat'
mat = scipy.io.loadmat(link_folder_cm006_unit4)

mat['cfuInfo1'].shape

mat.keys()

list(mat.keys())[3]

df_test = {}
mask_name = 'Mask_'
for i in range(13):
    df_test[i] = Image.fromarray((mat['cfuInfo1'][i][2] / np.max( mat['cfuInfo1'][i][2]) * 255).astype(np.uint8))

In [ ]:
mat = scipy.io.loadmat(link_file_mat)
cfu_to_tif_global(link_file_mat, link_save, list(mat.keys())[3], 2)
pd_data, pd_data_df = get_data_global(link_file_mat, link_folder + '\Save', list(mat.keys())[3])
pd_data_df = pd.DataFrame(pd_data_df)
pd_data = pd.DataFrame(pd_data)
pd_data_corr = pd_data-110
pd_data_corr_baseline = calculate_baseline(pd_data_corr, sigma=10, deg=5)
pd_data_corr_df =pd.DataFrame()
for col in pd_data_corr_baseline .columns:
    pd_data_corr_df[col] = (pd_data_corr[col]-pd_data_corr_baseline[col])/pd_data_corr_baseline[col]

pd_data_corr_df_baselinetocheck = calculate_baseline(pd_data_corr_df, sigma=10, deg=5)
plot_and_save(pd_data_corr_df, pd_data_corr_df_baselinetocheck, link_folder + '\Save' + '\corrected_baseline')
peak_indices_df, peak_counts_df = detect_peaks(
    df=pd_data_corr_df,
    height=0.1,        # Minimum height of peaks (adjust as needed)
    prominence=0.08,   # Minimum prominence of peaks
    distance=30        # Minimum distance between peaks
)

print(peak_counts_df)
plot_peaks(pd_data_corr_df, peak_indices_df, link_folder + '\Save' + '\CFU_Peaks_calc')

In [ ]:
df = pd_data_corr_df
peak_indices, smoothed_data, peak_counts = detect_peaks_on_smoothed_data(df, fs=30.9, cutoff=5, height=0.1, prominence=0.2, distance=15)
print(peak_counts)
time = np.linspace(0, len(df[cfu]) / 30.9, len(df[cfu]))
cfu = "Mask_1"
original_data = df[cfu].values
smoothed = smoothed_data[cfu]
peaks = peak_indices[cfu]

fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=original_data, mode='lines', name="Original Data", line=dict(color="blue", width=1)))
fig.add_trace(go.Scatter(x=time, y=smoothed, mode='lines', name="Smoothed Data", line=dict(color="green", width=2)))
fig.add_trace(go.Scatter(x=time[peaks], y=smoothed[peaks], mode='markers', name="Peaks", marker=dict(color='red', size=8)))
fig.update_layout(
    title=f"Peaks for {cfu}",
    xaxis_title="Time (s)",
    yaxis_title="Intensity",
    template="plotly_white",
    legend=dict(x=1, y=1)
)
fig.show()



In [ ]:
cfu = "Mask_1"
original_data = df[cfu].values
smoothed = smoothed_data[cfu]
peaks = peak_indices[cfu]

fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=original_data, mode='lines', name="Original Data", line=dict(color="blue", width=1)))
fig.add_trace(go.Scatter(x=time, y=smoothed, mode='lines', name="Smoothed Data", line=dict(color="green", width=2)))
fig.add_trace(go.Scatter(x=time[peaks], y=smoothed[peaks], mode='markers', name="Peaks", marker=dict(color='red', size=8)))
fig.update_layout(
    title=f"Peaks for {cfu}",
    xaxis_title="Time (s)",
    yaxis_title="Intensity",
    template="plotly_white",
    legend=dict(x=1, y=1)
)
fig.show()

In [ ]:
output_dir = r'E:\CM006_uni4_test_matlab_aqua_CFUs\unit4_crop2_32\Save' + '\CFU_Peaks_calc_smooth'
plot_peaks_on_smoothed_data_with_baseline(df, smoothed_data, peak_indices, pd_data_corr_df_baselinetocheck, output_dir)


In [ ]:

plot_peak_density_per_cfu(peak_indices, fs=30.9, output_dir=r'E:\CM006_uni4_test_matlab_aqua_CFUs\unit4_crop2_32\Save' + '\CFU_Peaks_dencity' ,bins=20)

# Script to run for the data analysis

How to use?

- get the tiff files from anywhere, better - suite2p corrected
- convert to 32 bit through ImageJ, crop to get the final size less than 4 Gb
- save the final file
- recommended to create a folder to place the file
- run the AQuA2 in Matlab (no excluded borders, set the fps etc.), threshold on 2-3, test a few)
- run CFU and choose the CFUs that you need
- load the traces to the workspace and save the .mat file with the masks and traces to the previously created folder

Then:
- set up the standard conda env and install all necessary libs
- run the imports
- run the functions
- create the cell with the links to the .mat file and where to save the outputs
- run the cell with links and the script cell

# Links (current)

In [ ]:
# CM006 Unit4 crop 2
link_folder = r'E:\Data\241105_CM006\unit4'
link_file_mat = r'E:\Data\241105_CM006\unit4\unit4_32_crop_2_AQuA2.mat'
link_save = r'E:\Data\241105_CM006\unit4\Save'
sigma_value = 10
deg_value = 5
fs_value = 30.9
height_value=0.2        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

In [ ]:
# CM005 Unit10 
link_folder = r'E:\CM005_unit10'
link_file_mat = r'E:\CM005_unit10\cm005_unit10.mat'
link_save = r'E:\CM005_unit10\Save'
sigma_value = 10
deg_value = 5
fs_value = 54
height_value=0.2        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

In [ ]:
# CM006 Unit8 
link_folder = r'E:\Data\241105_CM006\unit8'
link_file_mat = r'E:\Data\241105_CM006\unit8\unit8_32_bottomhalf.mat'
link_save = r'E:\Data\241105_CM006\unit8\Save'
sigma_value = 10
deg_value = 5
fs_value = 30.9
height_value=0.3        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

In [ ]:
# CM005 Unit5 
link_folder = r'E:\Data\241106_CM005\unit5'
link_file_mat = r'E:\Data\241106_CM005\unit5\unit5_32_corr.mat'
link_save = r'E:\Data\241106_CM005\unit5\Save'
sigma_value = 10
deg_value = 5
fs_value = 30.9
height_value=0.2        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

# Data

In [29]:
# 241202_CM005_rec1_more_unit0_32_crop
link_folder = r'E:\Data\analysis_cm005'
link_file_mat = r'E:\Data\analysis_cm005\241202_CM005_rec1_more_unit0_32_crop.mat'
link_save = r'E:\Data\analysis_cm005\Save_241202_CM005_rec1_more_unit0_32_crop'
sigma_value = 7
deg_value = 5
fs_value = 30.9
height_value=0.2        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

cols_list = ['Mask_0','Mask_50', 'Mask_61', 'Mask_43']
cfu_list = ['CFU_0','CFU_50', 'CFU_61', 'CFU_43']

In [30]:
# 241202_CM005_rec1_unit4_32_all
link_folder = r'E:\Data\analysis_cm005'
link_file_mat = r'E:\Data\analysis_cm005\241202_CM005_rec1_unit4_32_all.mat'
link_save = r'E:\Data\analysis_cm005\Save_241202_CM005_rec1_unit4_32_all'
sigma_value = 10
deg_value = 5
fs_value = 61
height_value=0.2        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

cols_list = ['Mask_0','Mask_95', 'Mask_99', 'Mask_110','Mask_115', 'Mask_117', 'Mask_174']
#cfu_list = ['CFU_0','CFU_50', 'CFU_61', 'CFU_43']

In [90]:
# 241202_CM005_rec2_unit5_32_crop_AQuA2
link_folder = r'E:\Data\analysis_cm005'
link_file_mat = r'E:\Data\analysis_cm005\241202_CM005_rec2_unit5_32_crop_AQuA2.mat'
link_save = r'E:\Data\analysis_cm005\Save_241202_CM005_rec2_unit5_32_crop_AQuA2'
sigma_value = 10
deg_value = 5
fs_value = 61.8
height_value=0.2        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

cols_list = ['Mask_0','Mask_10','Mask_11', 'Mask_12', 'Mask_28','Mask_31']
#cfu_list = ['CFU_0','CFU_50', 'CFU_61', 'CFU_43']

In [71]:
# 241106_CM005_unit5_32_corr_upd
link_folder = r'E:\Data\analysis_cm005'
link_file_mat = r'E:\Data\analysis_cm005\241106_CM005_unit5_32_corr_upd.mat'
link_save = r'E:\Data\analysis_cm005\Save_241106_CM005_unit5_32_corr_upd'
sigma_value = 10
deg_value = 5
fs_value = 30.9
height_value=0.15        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

cols_list = ['Mask_0','Mask_3', 'Mask_4', 'Mask_7','Mask_6','Mask_14', 'Mask_15', 'Mask_16','Mask_17', 'Mask_18','Mask_20']
#cfu_list = ['CFU_0','CFU_50', 'CFU_61', 'CFU_43']

In [111]:
# 241106_CM005_unit10_32_upd
link_folder = r'E:\Data\analysis_cm005'
link_file_mat = r'E:\Data\analysis_cm005\241106_CM005_unit10_32_upd.mat'
link_save = r'E:\Data\analysis_cm005\Save_241106_CM005_unit10_32_upd'
sigma_value = 10
deg_value = 5
fs_value = 52
height_value=0.3        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

cols_list = ['Mask_0','Mask_48', 'Mask_51', 'Mask_54','Mask_62']
#cfu_list = ['CFU_0','CFU_50', 'CFU_61', 'CFU_43']

In [124]:
# 241106_CM005_unit9_32_upd
link_folder = r'E:\Data\analysis_cm005'
link_file_mat = r'E:\Data\analysis_cm005\241106_CM005_unit9_32_upd.mat'
link_save = r'E:\Data\analysis_cm005\Save_241106_CM005_unit9_32_upd'
sigma_value = 10
deg_value = 5
fs_value = 52
height_value=0.3        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

cols_list = ['Mask_0','Mask_48', 'Mask_51', 'Mask_54','Mask_62']
#cfu_list = ['CFU_0','CFU_50', 'CFU_61', 'CFU_43']

In [125]:
mat = scipy.io.loadmat(link_file_mat)
cfu_to_tif_global(link_file_mat, link_save, list(mat.keys())[3], 2)
pd_data, pd_data_df = get_data_global(link_file_mat, link_save, list(mat.keys())[3])
pd_data_df = pd.DataFrame(pd_data_df)
pd_data = pd.DataFrame(pd_data)
pd_data_corr = pd_data-110

In [126]:
pd_data_corr = bin_dataframe(pd_data_corr, bin_factor=2)
fs_value = fs_value/2

In [127]:
pd_data_corr_baseline = calculate_baseline(pd_data_corr, sigma=sigma_value, deg = deg_value)
pd_data_corr_df = pd.DataFrame()
for col in pd_data_corr_baseline .columns:
    pd_data_corr_df[col] = (pd_data_corr[col]-pd_data_corr_baseline[col])/pd_data_corr_baseline[col]
pd_data_corr_df_baselinetocheck = calculate_baseline(pd_data_corr_df, sigma=sigma_value, deg = deg_value)

In [121]:
pd_data_df = pd_data_df[cols_list]
pd_data_corr = pd_data_corr[cols_list]
pd_data_corr_df = pd_data_corr_df[cols_list]
pd_data_corr_df_baselinetocheck = pd_data_corr_df_baselinetocheck[cols_list]
pd_data_corr_baseline = pd_data_corr_baseline[cols_list]


KeyError: "['Mask_48', 'Mask_51', 'Mask_54', 'Mask_62'] not in index"

In [128]:
plot_and_save(pd_data_corr_df, pd_data_corr_df_baselinetocheck, link_save + '\corrected_baseline')
output_dir = link_save + '\CFU_Peaks_calc_smooth'
peak_indices, smoothed_data, peak_counts = detect_peaks_on_smoothed_data(pd_data_corr_df, 
                                                                         fs=fs_value, 
                                                                         cutoff=cutoff_value, 
                                                                         height=height_value,        # Minimum height of peaks (adjust as needed)
                                                                         prominence=prominence_value,   # Minimum prominence of peaks
                                                                         distance=distance_value )
print(peak_counts)
plot_peaks_on_smoothed_data_with_baseline(pd_data_corr_df, 
                                          smoothed_data, peak_indices, 
                                          pd_data_corr_df_baselinetocheck, 
                                          link_save + '\CFU_Peaks_calc_smooth', fs_value)
plot_peak_density_per_cfu(peak_indices, fs=fs_value, output_dir=link_save + '\CFU_Peaks_dencity')
save_cfu_analysis_with_report_v2(
    pd_data_corr_df, smoothed_data, pd_data_corr_df_baselinetocheck, peak_indices, link_save,
    link_folder, link_file_mat, link_save,
    sigma_value, deg_value, fs_value, height_value, prominence_value, distance_value)
plot_combined_smoothed_and_peak_density(
     pd_data_corr_df, smoothed_data, peak_indices, fps=fs_value, 
     height_value=height_value, prominence_value=prominence_value, distance_value=distance_value, sigma_value=sigma_value,
     output_path=link_save)
overlap_df = compare_peak_times(
     peak_indices, tolerance_frames=1, fps=fs_value, output_path=link_save
 )


Saved interactive graph for CFU Mask_0 as E:\Data\analysis_cm005\Save_241106_CM005_unit9_32_upd\corrected_baseline\cfu_Mask_0_graph.html
Saved interactive graph for CFU Mask_1 as E:\Data\analysis_cm005\Save_241106_CM005_unit9_32_upd\corrected_baseline\cfu_Mask_1_graph.html
Saved interactive graph for CFU Mask_2 as E:\Data\analysis_cm005\Save_241106_CM005_unit9_32_upd\corrected_baseline\cfu_Mask_2_graph.html
Saved interactive graph for CFU Mask_3 as E:\Data\analysis_cm005\Save_241106_CM005_unit9_32_upd\corrected_baseline\cfu_Mask_3_graph.html
Saved interactive graph for CFU Mask_4 as E:\Data\analysis_cm005\Save_241106_CM005_unit9_32_upd\corrected_baseline\cfu_Mask_4_graph.html
Saved interactive graph for CFU Mask_5 as E:\Data\analysis_cm005\Save_241106_CM005_unit9_32_upd\corrected_baseline\cfu_Mask_5_graph.html
Saved interactive graph for CFU Mask_6 as E:\Data\analysis_cm005\Save_241106_CM005_unit9_32_upd\corrected_baseline\cfu_Mask_6_graph.html
Saved interactive graph for CFU Mask_7 as

sigma_value = 10
deg_value = 5
fs_value = 30.9
height_value=0.2        # Minimum height of peaks (adjust as needed)
prominence_value=0.1   # Minimum prominence of peaks
distance_value=2
cutoff_value = 5

In [95]:
plot_high_overlap_pairs(pd_data_corr_df, smoothed_data, peak_indices, overlap_df, 20, fs_value, link_save)

Saved high-overlap plot for Mask_10 and Mask_12 to E:\Data\analysis_cm005\Save_241202_CM005_rec2_unit5_32_crop_AQuA2\overlap_Mask_10_Mask_12.html
Saved high-overlap plot for Mask_11 and Mask_12 to E:\Data\analysis_cm005\Save_241202_CM005_rec2_unit5_32_crop_AQuA2\overlap_Mask_11_Mask_12.html
Saved high-overlap plot for Mask_12 and Mask_10 to E:\Data\analysis_cm005\Save_241202_CM005_rec2_unit5_32_crop_AQuA2\overlap_Mask_12_Mask_10.html
Saved high-overlap plot for Mask_12 and Mask_11 to E:\Data\analysis_cm005\Save_241202_CM005_rec2_unit5_32_crop_AQuA2\overlap_Mask_12_Mask_11.html
